In [ ]:
import os
import urllib3
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'

mlflow.set_tracking_uri("https://mlflow.labs.itmo.loc")

try:
    mlflow.set_experiment("Lab4-JupyterHub-Training")
    print("Эксперимент настроен")
except Exception as e:
    print(f"Не удалось настроить эксперимент: {e}")

print("Начинаем обучение модели...")
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run(run_name="Training-With-Model-Registry") as run:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("dataset", "synthetic_classification")
    mlflow.log_param("samples", X.shape[0])
    mlflow.log_param("features", X.shape[1])
    
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("accuracy_difference", train_accuracy - test_accuracy)
    
    print(f"Обучение завершено! Точность: {test_accuracy:.4f}")
    mlflow.set_tags({
        "environment": "jupyterhub", 
        "purpose": "lab4-training",
        "model_type": "random_forest",
        "registered": "true"
    })
    
    print(f"Run ID: {run.info.run_id}")

In [ ]:
import os
import urllib3
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'

mlflow.set_tracking_uri("https://mlflow.labs.itmo.loc")

client = MlflowClient()
run_id = "1750b23f64854374ab838f4e967cbec6"
model_name = "Lab4-Classification-Model"

try:
    try:
        client.get_registered_model(model_name)
        print(f"Модель {model_name} уже существует")
    except:
        client.create_registered_model(model_name)
        print(f"Создана новая модель: {model_name}")
    
    result = client.create_model_version(
        name=model_name,
        source=f"runs:/{run_id}/model",
        run_id=run_id
    )
    
    print(f"Создана версия модели: {result.version}")
    
    client.transition_model_version_stage(
        name=model_name,
        version=result.version,
        stage="Production"
    )
    
    print("Модель переведена в stage: Production")
    print(f"Модель доступна по имени: {model_name}")
    
except Exception as e:
    print(f"Ошибка регистрации: {e}")

In [ ]:
# В JupyterHub выполните это
import mlflow
mlflow.set_tracking_uri("https://mlflow.labs.itmo.loc")

# Проверим run
run = mlflow.get_run("1750b23f64854374ab838f4e967cbec6")
print("Run info:")
print(f"Status: {run.info.status}")
print(f"Artifact URI: {run.info.artifact_uri}")

# Проверим артефакты
client = mlflow.tracking.MlflowClient()
artifacts = client.list_artifacts(run.info.run_id)
print("Artifacts:")
for artifact in artifacts:
    print(f" - {artifact.path}")

In [23]:
# В JupyterHub
import mlflow
import mlflow.sklearn
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os
import tempfile
from mlflow.tracking import MlflowClient

# Настройка MLflow
os.environ['MLFLOW_TRACKING_INSECURE_TLS'] = 'true'
mlflow.set_tracking_uri("https://mlflow.labs.itmo.loc")
mlflow.set_experiment("Lab4-JupyterHub-Training")

print("🔄 Создаем модель с правильным сохранением...")

# Обучение модели
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with mlflow.start_run(run_name="Working-Model-With-Artifacts") as run:
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Логируем параметры
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("features", 20)
    
    # Логируем метрики
    train_accuracy = model.score(X_train, y_train)
    test_accuracy = model.score(X_test, y_test)
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("test_accuracy", test_accuracy)
    
    print(f"📊 Точность: {test_accuracy:.4f}")
    
    # ✅ Сохраняем модель во временную директорию
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model")
        mlflow.sklearn.save_model(model, model_path)
        
        # ✅ Логируем артефакты из временной директории
        mlflow.log_artifacts(model_path, "model")
    
    print(f"✅ Модель сохранена как артефакт!")
    print(f"🎯 Run ID: {run.info.run_id}")

# Проверим артефакты
client = MlflowClient()
artifacts = client.list_artifacts(run.info.run_id)
print("\n📁 Артефакты:")
for artifact in artifacts:
    print(f" - {artifact.path}")

# Создаем новую версию модели
try:
    # Создаем новую версию
    result = client.create_model_version(
        name="Lab4-Classification-Model",
        source=f"runs:/{run.info.run_id}/model",
        run_id=run.info.run_id
    )
    
    print(f"✅ Новая версия создана: {result.version}")
    
    # Переводим в Production
    client.transition_model_version_stage(
        name="Lab4-Classification-Model",
        version=result.version,
        stage="Production"
    )
    
    print("✅ Версия переведена в Production")
    
except Exception as e:
    print(f"⚠️ Ошибка создания версии: {e}")

🔄 Создаем модель с правильным сохранением...
📊 Точность: 0.9000
🏃 View run Working-Model-With-Artifacts at: https://mlflow.labs.itmo.loc/#/experiments/4/runs/dfcf234131e24e538985f951a63fa612
🧪 View experiment at: https://mlflow.labs.itmo.loc/#/experiments/4


PermissionError: [Errno 13] Permission denied: '/home/user'